In [ ]:
from wave_client import WaveClient
import pandas as pd
from utils import (
    load_environment_variables,
)


In [12]:
# Load environment variables and connect to WAVE backend
RESEARCHER_API_KEY, EXPERIMENTEE_API_KEY, WAVE_BACKEND_URL = load_environment_variables()

print(f"🔍 Connecting to WAVE backend...")
print(f"📡 Backend URL: {WAVE_BACKEND_URL}")
print(f"🔑 Using Researcher API key ending in: ...{RESEARCHER_API_KEY[-4:]}")

client = WaveClient(api_key=RESEARCHER_API_KEY, base_url=WAVE_BACKEND_URL)

Present working directory: c:\Users\inkpe\local-JS-Williams\shadows_3afc\tools
Loading environment variables from c:\Users\inkpe\local-JS-Williams\shadows_3afc\tools/.env
🔍 Connecting to WAVE backend...
📡 Backend URL: https://wave-backend-production-8781.up.railway.app
🔑 Using Researcher API key ending in: ...SYuP


In [ ]:
## use this (uncomment it) to see what commands exist 
## (allow the scrollable option at the bottom to see all commands)

# help(client.experiments)
# help(client.experiment_data)

Help on ExperimentsResource in module wave_client.resources.experiments object:

class ExperimentsResource(wave_client.resources.base.BaseResource)
 |  ExperimentsResource(client: 'WaveClient')
 |
 |  Resource for managing experiments.
 |
 |  Method resolution order:
 |      ExperimentsResource
 |      wave_client.resources.base.BaseResource
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  async create(self, experiment_type_id: int, description: str, tags: Optional[List[str]] = None, additional_data: Optional[Dict[str, Any]] = None) -> Dict[str, Any]
 |      Create a new experiment.
 |
 |      Args:
 |          experiment_type_id: Experiment type ID (must be > 0).
 |          description: Experiment description.
 |          tags: List of tags (max 10).
 |          additional_data: Additional metadata.
 |
 |      Returns:
 |          Created experiment response.
 |
 |  async delete(self, experiment_uuid: str) -> Dict[str, Any]
 |      Delete experiment (requires ADMIN role).


In [ ]:
# Get all experiments to see which one to analyze
experiments = await client.experiments.list(skip=0, limit=200)
print(f"Found {len(experiments)} experiments\n")

# Display experiment info
for i, exp in enumerate(experiments, 1):
    print(f"{i}. Experiment UUID: {exp['uuid']}")
    print(f"   Description: {exp.get('description', 'N/A')}")
    print(f"   Type ID: {exp.get('experiment_type_id', 'N/A')}")
    print(f"   Created: {exp.get('created_at', 'N/A')[:10]}")
    print(f"   Tags: {', '.join(exp.get('tags', []))}")
    print("-" * 70)


Found 3 experiments

1. Experiment UUID: 0ef3dc5a-b6d2-4c17-90ee-e83880a15c5f
   Description: shadow_afc_prod
   Type ID: 6
   Created: 2026-01-08
   Tags: 
----------------------------------------------------------------------
2. Experiment UUID: d4b9662a-5019-4a0b-881c-1e761feb2119
   Description: Test experiment for jspsych_color_circles_demo - TEST_jspsych_color_circles_demo_20260108_223942
   Type ID: 7
   Created: 2026-01-09
   Tags: test
----------------------------------------------------------------------
3. Experiment UUID: 56332cb3-b54a-4828-be3e-bbc786118885
   Description: jspsych_circles_prod
   Type ID: 7
   Created: 2026-01-09
   Tags: 
----------------------------------------------------------------------


In [ ]:
# Put the necessary info into the variables below 
selected_experiment_uuid = "0ef3dc5a-b6d2-4c17-90ee-e83880a15c5f" 

# Get all data from that experiment
print(f"\n📊 Fetching data for experiment {selected_experiment_uuid}...")
experiment_data_df = await client.experiment_data.get_all_data(
    experiment_id=selected_experiment_uuid
)

print(f"Total records: {len(experiment_data_df)}")
# print(experiment_data_df)



📊 Fetching data for experiment 0ef3dc5a-b6d2-4c17-90ee-e83880a15c5f...
Total records: 15
   response  response_time
5       3.0         2011.0
0       1.0         5883.0


In [ ]:
# Filter and save into .csv file
full_df = (experiment_data_df.query("participant_id != '12345678'")
           .sort_values(by="timestamp",ascending=True))
full_df.to_csv("data/dataframe_full.csv", index = False)

answer_df = full_df.query("trial_category == 'afcexpt'")
answer_df.to_csv("data/dataframe_answer.csv", index = False)



In [32]:
# Example of how to pull out only a few specific columns

selected_columns = answer_df[["response","response_time","chosen_image","target_object","target_shadow"]]
print(selected_columns)

   response  response_time chosen_image target_object target_shadow
5       3.0         2011.0    objC_shaC             A             D
0       1.0         5883.0    objB_shaB             B             A
